<a href="https://colab.research.google.com/github/ananthakrishnanpta/Audio-classification-project-DS/blob/main/speech_recognition_and_speaker_diarization_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependancies :- 

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

!pip install PyAudio

!pip install pydub

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 0s (585 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 156210 files and directories currently installed.)
Preparing to 

In [ ]:
# Giving file path and api key for testing purposes

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Colab Notebooks/audio project/test_file_StT.m4a'


api_key = "b91555dcfb9b45088a64295412976362" #limitation of this free api key is that only total 3 hours length of transcription can be done in a month.
# Now We are using the free 3 hour per month limit. If we take paid version, it is Billed at $0.00025 per second transcribed
session_id = 123

Mounted at /content/drive


# Transcription function

### -> Takes in file_path and api_key as inputs
### -> Returns transcript in a json dictionary
### -> Carries out speaker diarization and creates a separate transcript file for each identified speaker.

In [ ]:
def transcribe(audio_path, api_key):

  # An additional argument can be passed in as session_id or tutor_id 
  # so that it could be added to the final transcribed file name.

  import argparse
  print('1. API is read ...')
  
  # 2. Retrieving audio file from class video
  import os
  
  print('2. Audio file has been retrieved.')
  
  # 3. Upload class audio file to AssemblyAI
  import sys
  import time
  import requests
  
  filename = audio_path
  def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
      while True:
        data = _file.read(chunk_size)
        if not data:
          break
        yield data
  headers = {'authorization': api_key}
  response = requests.post('https://api.assemblyai.com/v2/upload',
                           headers=headers,
                           data=read_file(filename))
  
  audio_url = response.json()['upload_url'] #audio url for uploading audio to Assembly AI's hosting service
  
  print('3. Audio file has been uploaded to AssemblyAI')
  
  # 4. Transcribe uploaded audio file
  
  import requests
  
  endpoint = "https://api.assemblyai.com/v2/transcript"
  
  json = {
      "audio_url": audio_url,
      "sentiment_analysis" : True, #switching on sentimental analysis
      "speaker_labels": True #switching on the speaker diarization
      
      }
      
  headers = {
      "authorization": api_key,
      "content-type": "application/json"
      }
      
  transcript_input_response = requests.post(endpoint, json=json, headers=headers)
  
  print('4. Transcribing uploaded file')
  
  # 5. Extract transcript ID ...
  
  transcript_id = transcript_input_response.json()["id"]
  
  print('5. Extracting transcript ID')
  
  # 6. Retrieving transcription results ...
  
  endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
  headers = {
      "authorization": api_key,
      }
      
  transcript_output_response = requests.get(endpoint, headers=headers) #Retrieving the json formatted output from API
  
  print('6. Retrieve transcription results')
  
  # Check if transcription is complete
  
  from time import sleep
  
  # Transcription takes time and this while loop helps check the completion in defined intervals.
  while transcript_output_response.json()['status'] != 'completed':
    sleep(5)
    print('Transcription is processing ...')
    transcript_output_response = requests.get(endpoint, headers=headers)
    
  #print(transcript_output_response.json()["status"])
    
  # 7. Print transcribed text
    
  print('----------\n')
  print('Output:\n')
  print(transcript_output_response.json()["text"])
  data = transcript_output_response.json()
  
  # 8. Save transcribed text to file
  
  # Save as TXT file - right now, it is writing the entire text. 
  # We can select just the text with label of specified speaker from the json data and 
  # this is implemented later on
  
  transcript_txt = open('session_transcript.txt', 'w')
  transcript_txt.write(transcript_output_response.json()["text"])
  transcript_txt.close()
  
  # Save as SRT file -  could help for manual classification of the videos
  srt_endpoint = endpoint + "/srt"
  srt_response = requests.get(srt_endpoint, headers=headers)
  
  with open("session_transcript.srt", "w") as _file:
    _file.write(srt_response.text)

  speaker_list = []
  
  for word in data['words']:
    speaker_list.append(word['speaker'])
  
  # Creating a list of uniquely identified speakers :- 
  
  speaker_list = list(set(speaker_list))
  
  print("Identified speakers :- ",speaker_list)

  # Creating a separate transcript for each identified speaker 

  for speaker in speaker_list:
    # currently the name of transcript file is formatted in this way :- "<speaker_label>--transcribed.txt", 
    # session_id could be added to help while transcription is automated.
    
    with open(speaker + "--transcribed.txt", "a") as text_file:
      for i in transcript_output_response.json()['words']:
        if i['speaker'] == speaker:
          print(i['text'], end = " ",file = text_file)

  # Creating a dataframe of sentiment analysis results:- 
  import pandas as pd

  sentiment_df = pd.DataFrame.from_dict(transcript_output_response.json()['sentiment_analysis_results'])

  # Renaming columns 
  sentiment_df.rename(columns = {'text':'Text','start':'Start_time(ms)', 'end':'End_time(ms)','sentiment':'Sentiment','confidence':'Confidence','speaker':'Participant'},inplace = True)

  # Reordering Column names
  sentiment_df = sentiment_df[['Participant','Sentiment','Text','Confidence','Start_time(ms)','End_time(ms)']]

  # Writing dataframe to excel sheet



  sentiment_df.to_excel("sentiment_output.xlsx")
  return sentiment_df


In [ ]:
data = transcribe(file_path, api_key )

1. API is read ...
2. Audio file has been retrieved.
3. Audio file has been uploaded to AssemblyAI
4. Transcribing uploaded file
5. Extracting transcript ID
6. Retrieve transcription results
Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
----------

Output:

Hello, Jamie. Your mic is on mute. Yeah. Hi Jamie. How are you? Well, I'm good. How are you doing? I'm doing fine. So you've completed your first file? I did. I just had so many classes, I don't know where I'm supposed to be. Okay, so like when you are supposed to connect, your name will be there. It will be like, Hi Jamir, please join the meeting. If the message is like that, you can join in. If the message is like, what? Sorry, li

In [ ]:
def highlight_cols(x):
  color = 'white'
  if x == 'POSITIVE':
    color = 'green'
  elif x == 'NEGATIVE':
    color = 'red'

  return 'background-color: %s' %color

df_new = data.style.applymap(highlight_cols)
df_new.to_excel('color_formatted_output.xlsx')
display(df_new)

,Participant,Sentiment,Text,Confidence,Start_time(ms),End_time(ms)
0,A,NEUTRAL,"Hello, Jamie.",0.664225,9070,10726
1,A,NEUTRAL,Your mic is on mute.,0.543702,10798,12210
2,B,NEUTRAL,Yeah.,0.549145,14370,14986
3,A,NEUTRAL,Hi Jamie.,0.705399,15108,15662
4,A,NEUTRAL,How are you?,0.734534,15686,16500
5,B,POSITIVE,"Well, I'm good.",0.896210,17550,18934
6,B,NEUTRAL,How are you doing?,0.792130,19032,20038
7,A,POSITIVE,I'm doing fine.,0.832110,20184,21262
8,A,NEUTRAL,So you've completed your first file?,0.882132,21396,23590
9,B,NEUTRAL,I did.,0.596080,24810,25522


In [ ]:
df_new.columns

Index(['Participant', 'Sentiment', 'Text', 'Confidence', 'Start_time(ms)',
       'End_time(ms)'],
      dtype='object')

In [ ]:
data

,Participant,Sentiment,Text,Confidence,Start_time(ms),End_time(ms)
0,A,NEUTRAL,"Hello, Jamie.",0.664225,9070,10726
1,A,NEUTRAL,Your mic is on mute.,0.543702,10798,12210
2,B,NEUTRAL,Yeah.,0.549145,14370,14986
3,A,NEUTRAL,Hi Jamie.,0.705399,15108,15662
4,A,NEUTRAL,How are you?,0.734534,15686,16500
...,...,...,...,...,...,...
94,B,NEUTRAL,Talk to you soon then.,0.774425,293790,295080
95,A,NEUTRAL,Yeah.,0.549145,295470,296220
96,B,NEUTRAL,All right.,0.489670,296610,297034
97,B,NEUTRAL,Bye for now.,0.547880,297072,297886
